# 2. 在交易策略中使用深度学习模型进行交易决策

在这个 Notebook 中我们将尝试训练一个简单的自定义序贯（Sequential）模型，用于判断是否应该在技术指标出现交易信号的时候进行交易。通常来讲，技术指标是基于数学公式所计算出的指标值。技术指标会随行情变化而变化。技术指标可以更直接地反映股市所处的状态，为交易提供指导。比如说，我们在之前的 BackTrader 策略示例中就以收盘价上穿均线作为开仓条件，收盘价下穿均线作为平仓条件。

然而，通过指标发出的买卖信号是一个随机过程，不可能非常准确。很多技术指标容易产生钝化，或发出一些错误的买卖信号。在本示例中，我们将设定一些，并通过机器学习模型做进一步的预测和判断。您可以通过回测来观察这样做是否能够改善交易结果。

完成这个实验需要 TensorFlow 框架，请选择 conda_amazonei_tensorflow2_p36 环境。

在开始之前，可以首先升级环境自带的 TensorFlow 和 Keras 最新版本：

In [ ]:
!pip install --upgrade tensorflow keras
!pip show tensorflow keras

接下来安装实验需要的一些依赖包：

In [ ]:
!pip install --upgrade pip
!pip install backtrader
!pip install matplotlib==3.1.3
!pip install talib-binary

最后安装火币 API 并重启内核：

In [ ]:
!git clone https://github.com/HuobiRDCenter/huobi_Python
directory = '/home/ec2-user/SageMaker/sagemaker-huobi-workshop'
!cd {directory}/huobi_Python && python3 setup.py -q install
!pip show huobi-client
import os

os._exit(00)

## 准备工作

在这个部分我们将进行数据的准备工作、训练必要的模型并编写策略的回测脚本。

首先，应当确保环境变量中有 SageMaker 实例的默认路径：

In [ ]:
import sys

directory = '/home/ec2-user/SageMaker/sagemaker-huobi-workshop'  # '/root/sagemaker-huobi-workshop' for SageMaker Studio
if directory not in sys.path:
    print(directory, 'added to sys.path')
    sys.path.append(directory)
    
prefix = 'model_training'

# 创建输入和输出路径
import pathlib

pathlib.Path(directory + '/' + prefix + '/input/data').mkdir(parents=True, exist_ok=True)
pathlib.Path(directory + '/' + prefix + '/output').mkdir(parents=True, exist_ok=True)

## 定义超参

在开始之前我们先定义一下这个算法中包含几个重要的参数：
  - long_threshold：当模型预测止盈概率超过此数值时做多 (0 到 1 之间的小数)。
  - short_threshold：当模型预测止损概率超过此数值时做空 (0 到 1 之间的小数)。如果设为大于等于 1，策略就不会做空。
  - profit_target：当交易盈利超过此比例时止盈（0 到 1 之间的小数）。
  - stop_target：当交易损失超过此比例时止损（0 到 1 之间的小数）。
  - look_back：每次预测使用过去多少个交易日（整数）的历史数据。（look_back = repeat_count * repeat_step）
  - forward_window：每次预测向前推导多少个交易日（整数）。

In [ ]:
# 默认参数
params = { 
    "long_threshold" : 0.5,
    "short_threshold" : 1,
    "profit_target" : 0.02,
    "stop_target" : 0.01,
    "repeat_count": 20,
    "repeat_step": 1,
    "forward_window": 10
}

long_threshold = params['long_threshold']
short_threshold = params['short_threshold']
profit_target = params['profit_target']
stop_target = params['stop_target']

repeat_count = params['repeat_count']
repeat_step = params['repeat_step']
look_back = repeat_count * repeat_step
forward_window = params['forward_window']

# 将参数以 json 格式保存至指定目录
import json
with open('{}/hyperparameters.json'.format(directory), 'w') as fp:
    json.dump(params, fp)

## 模型训练

在这个演示中，我们将多获取自2010年至今的数据用于模型训练和预测。首先我们 import 必要的依赖包：

In [ ]:
import boto3
import datetime
import numpy as np
import pandas as pd
import pytz
import talib as ta
import time

### 训练数据

以下的代码将通过火币 API 调取必要的数据：

In [ ]:
from huobi.client.market import MarketClient
from huobi.constant import *
from huobi.utils import *

market_client = MarketClient(init_log=True)
interval = CandlestickInterval.DAY1
symbol = "btcusdt"

flag = True
while flag:
    try:
        list_obj = market_client.get_candlestick(symbol, interval, 1300)
        # LogInfo.output("---- {interval} candlestick for {symbol} ----".format(interval=interval, symbol=symbol))
        # LogInfo.output_list(list_obj)
        flag = False
        print('Data load success')
    except:
        continue

将调取的火币原始数据转换为 Pandas DataFrame：

In [ ]:
columns = ['tradedate', 'high', 'low', 'open', 'close', 'count', 'amount', 'volume']
df = pd.DataFrame([[i.id, i.high, i.low, i.open, i.close, i.count, i.amount, i.vol] for i in list_obj], columns=columns)
# convert id timestamp to datetime
timezone = pytz.timezone('Asia/Shanghai')
df['tradedate'] = df['tradedate'].apply(lambda x: datetime.datetime.fromtimestamp(x).astimezone(timezone).strftime('%Y-%m-%d'))
df.set_index('tradedate', inplace=True)
df.sort_index(inplace=True)
output_path = '{}/{}/input/data/data_raw.csv'.format(directory, prefix)
print('Location:', output_path)
df.to_csv(output_path)

start_date = df.index[0]
end_date = df.index[-1]
print('Sample range:', start_date, '-', end_date)

df.head()

### 模型训练

以下是我们实现准备的 Sequential 模型训练程序。该模型训练过程将执行 100 个 epoch。随着 epoch 数量的增加，应该能看到 loss 的持续下降以及 accuracy 的提升：

In [ ]:
from __future__ import print_function

import datetime
import json
import math
import numpy as np
import os
import pandas as pd
import sys
import talib as ta
import traceback

import tensorflow as tf
from keras.layers import Dropout, Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.models import Sequential
from keras.wrappers.scikit_learn import KerasRegressor


# Optional
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

config_path = '{}/'.format(directory)
input_path = '{}/{}/input/data/data_train.csv'.format(directory, prefix)
raw_path = '{}/{}/input/data/data_raw.csv'.format(directory, prefix)
train_path = '{}/{}/input/data/data_train.csv'.format(directory, prefix)
test_path = '{}/{}/input/data/data_test.csv'.format(directory, prefix)
output_path = '{}/{}/output'.format(directory, prefix)
model_path = '{}/model.h5'.format(directory)


# Process and prepare the data
def get_data(params):
    
    # get and save hyperparameters
    long_threshold = float(params['long_threshold'])
    short_threshold = float(params['short_threshold'])
    profit_target = float(params['profit_target'])
    stop_target = float(params['stop_target'])

    repeat_count = int(params['repeat_count'])
    repeat_step = int(params['repeat_step'])
    look_back = repeat_count * repeat_step
    forward_window = int(params['forward_window'])
    
    # read data from s3
    df = pd.read_csv(raw_path, index_col=0)
    closePrice = df["close"]

    # use talib to calculate SMA and ROC
    ## set header for transformed data
    header = ["tradedate", "close"]
    for i in range(0, repeat_count):
        header.append("sma" + str((i+1) * repeat_step))
    for i in range(0, repeat_count):
        header.append("roc" + str((i+1) * repeat_step))
    header.append("long")
    header.append("short")

    data = []

    ## SMA
    inputs = {'close': np.array(closePrice)}
    sma = []
    for i in range(0, repeat_count):
        sma.append(ta.SMA(np.array(closePrice), timeperiod=(i+1) * repeat_step + 1))
    ## ROC - Rate of change : ((price/prevPrice)-1)*100
    roc = []
    for i in range(0, repeat_count):
        roc.append(ta.ROC(np.array(closePrice), timeperiod=(i+1) * repeat_step + 1))

    ## count long and short
    long_count = 0
    short_count = 0
    n_count = 0
    n = 0
    for idx in df.index:
        if n < len(df) - forward_window - 1:
            idx_0 = idx
            close_price = df.loc[idx, 'close']
            temp = []
            temp.append(idx)

            temp2 = []
            temp2.append(close_price)

            # sma
            for i in range(0, repeat_count):
                if np.isnan(sma[i][n]):
                    temp2.append(close_price)
                else:
                    temp2.append(sma[i][n])

            min_value = min(temp2)
            max_value = max(temp2)
            for i in temp2:
                if max_value == min_value:
                    temp.append(0)
                else:
                    temp.append((i - min_value) / (max_value - min_value))

            for i in range(0, repeat_count):
                if np.isnan(roc[i][n]):
                    temp.append(0)
                else:
                    temp.append(roc[i][n])

            rClose = closePrice[(n+1):min(len(df)-1, n+1+forward_window)].values.tolist()
            min_value = min(rClose)
            max_value = max(rClose)

            # long condition
            if max_value >= close_price * (1+profit_target) and min_value >= close_price * (1-stop_target):
                long_count += 1
                temp.append(1)
            else:
                temp.append(0)

            # short condition
            if min_value <= close_price * (1-stop_target) and max_value <= close_price * (1+profit_target):
                short_count += 1
                temp.append(1)
            else:
                temp.append(0)

            data.append(temp)
            n += 1

    print("long：%s, short：%s" % (long_count,short_count))
    df2 = pd.DataFrame(data, columns=header)
    df2.set_index('tradedate', inplace=True)
    print('Range:', df2.index[0], '-', df2.index[-1])
    
    # save data
    df_train = df2.iloc[:800]
    print('Training set:', df_train.index[0], '-', df_train.index[-1])
    print('Location:', train_path)
    df_train.to_csv(train_path)

    df_test = df2.iloc[800:1000]
    print('Testing set:', df_test.index[0], '-', df_test.index[-1])
    print('Location:', test_path)
    df_test.to_csv(test_path)
    

# Process and prepare the data
def data_process(df, yLen, b):
    
    dataX = []
    dataY = []
    for idx, row in df.iterrows():
        row1 = []
        r = row[1:len(row)-yLen]
        for a in r:
            row1.append(a)
        x = np.array(row1, dtype=float)
        y = np.array(row[len(row)-yLen:], dtype=float)
        b = len(x)
        dataX.append(x)
        dataY.append(y)
        
    dataX = np.array(dataX)
    dataY = np.array(dataY)
    
    return dataX, dataY, b


def build_classifier(b, yLen):
    
    print("build_classifier:b=%s,yLen=%s" % (b, yLen))
    model = Sequential()
    model.add(Dense(b, input_dim=b, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(int(b/2), kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(yLen, kernel_initializer='normal', activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model


def generate_model(dataX, dataY, b, yLen):
    
    model = build_classifier(b, yLen)
    model.fit(dataX, dataY, epochs=100, batch_size=1)
    scores = model.evaluate(dataX, dataY, verbose=0)
    print("Training Data %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    
    return model


def train():
    
    try:
        with open("{}/hyperparameters.json".format(config_path)) as json_file:
            params = json.load(json_file)
        print('Parameter load success')
    except Exception as e:
        print(e)

    get_data(params)
    
    print('Starting the training.')
    
    yLen = 2
    b = 0
    
    try:
        
        df = pd.read_csv(input_path)
        dataX, dataY, b = data_process(df, yLen, b)
        print('b:', b, 'yLen:', yLen)
        model = generate_model(dataX, dataY, b, yLen)
        model.save(model_path)
        
        print('Training is complete. Model saved.')
        
        df = pd.read_csv(test_path)
        dataX, dataY, b = data_process(df, yLen, b)
        print('b:', b, 'yLen:', yLen)
        scores = model.evaluate(dataX, dataY, verbose=0)
        print("Test Data %s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
        
    except Exception as e:
        # Write out an error file. This will be returned as the failure
        # Reason in the DescribeTrainingJob result.
        trc = traceback.format_exc()
        with open(os.path.join(output_path, 'failure'), 'w') as s:
            s.write('Exception during training: ' + str(e) + '\n' + trc)
        # Printing this causes the exception to be in the training job logs
        print(
            'Exception during training: ' + str(e) + '\n' + trc,
            file=sys.stderr)
        # A non-zero exit code causes the training job to be marked as Failed.
        sys.exit(255)

        
if __name__ == '__main__':
    train()

    # A zero exit code causes the job to be marked a Succeeded.
    sys.exit(0)

看到 SystemExit: 0 则表明模型训练成功。

训练完成后的模型保存在 model/ 路径下，应该可以看到路径下有名为 model.h5 的模型文件生成。可以运行以下代码尝试在 TensorFlow 中加载模型：

In [ ]:
from keras.models import load_model

model_path = '{}/model.h5'.format(directory)
print('Model path:', model_path)
try:
    model = load_model(model_path)
    print('Model load success')
    
except Exception as e:
    print(e)

In [ ]:
# Summary of neural network
model.summary()

## 回测

接下来我们将定义运行回测任务所需的代码。

### 回测数据

在开始前我们先配置好回测所需的数据和参数：

In [ ]:
# 回测数据
df_backtest= df.iloc[1000:]
print('Backtest set:', df_backtest.index[0], '-', df_backtest.index[-1])
df_backtest.head()

# 超参
with open("{}/hyperparameters.json".format(directory)) as json_file:
    params = json.load(json_file)

### 策略定义

接下来的脚本包含了运行回测所需的策略代码。该策略将从示例的路径中加载之前训练好的模型。

In [ ]:
import backtrader as bt
import json
import math
import numpy as np
import pandas as pd
import talib as ta

import tensorflow as tf
import keras
from keras import backend as K
from keras.models import load_model


class MyStrategy(bt.Strategy):
    
    params = (
        ('model_path', ''), 
        ('long_threshold', 0.5), 
        ('short_threshold', 1), 
        ('profit_target', 0.02), 
        ('stop_target', 0.01), 
        ('repeat_count', 20), 
        ('repeat_step', 1), 
        ('printlog', True))

    def __init__(self):
        super(MyStrategy, self).__init__()

        self.order = None
        self.orderPlaced = False
                                
        self.model = load_model(self.params.model_path)
        
        # input / indicators
        self.long_threshold = self.params.long_threshold
        self.short_threshold = self.params.short_threshold
        self.repeat_count = self.params.repeat_count
        self.repeat_step = self.params.repeat_step
        self.profit_target = self.params.profit_target
        self.stop_target = self.params.stop_target
    
        self.sma=[]
        self.roc=[]
        for i in range(0, self.repeat_count):
            self.sma.append(bt.talib.SMA(self.data, timeperiod=(i+1)*self.repeat_step + 1, plot=False))
            self.roc.append(bt.talib.ROC(self.data, timeperiod=(i+1)*self.repeat_step + 1, plot=False))
        
    def next(self):
        super(MyStrategy, self).next()
        
        idx_0 = self.datas[0].datetime.datetime(0)
        close_price = self.datas[0].close
        temp = []
        
        temp2 = []
        temp2.append(close_price)

        ## sma
        for i in range(0, self.repeat_count):
            if math.isnan(self.sma[i][0]):
                temp2.append(close_price)
            else:
                temp2.append(self.sma[i][0])
                
        min_value = min(temp2)
        max_value = max(temp2)
        for i in temp2:
            if max_value == min_value:
                temp.append(0)
            else:
                temp.append((i - min_value) / (max_value - min_value))

        ## roc
        for i in range(0, self.repeat_count):
            if math.isnan(self.roc[i][0]):
                temp.append(0)
            else:
                temp.append(self.roc[i][0])
        
        ## dataX
        dataX = np.array([np.array(temp)])

        ## dataY
        dataY = self.model.predict(dataX)
        
        
        ## 开仓条件
        tLong = dataY[0][0]
#         tShort = dataY[0][1]
        if not self.position:
            fLong = (tLong > self.long_threshold) 
#             fShort = (tShort > self.short_threshold)
            if fLong:
                self.size = int(self.broker.cash / self.datas[0].close[0])
                self.order = self.buy(size=self.size)
                self.limitPrice = close_price + self.profit_target * close_price
                self.stopPrice = close_price - self.stop_target * close_price
#             elif fShort:
#                 self.order = self.sell(size=self.size)                
#                 self.limitPrice = close_price - self.profit_target * close_price
#                 self.stopPrice = close_price + self.stop_target * close_price

        ## 平仓逻辑
        if self.position:
            if self.position.size > 0:
                if close_price >= self.limitPrice or close_price <= self.stopPrice:
                    self.order = self.sell(size=self.size)
#             elif self.position.size < 0:
#                 if close_price <= self.limitPrice or close_price >= self.stopPrice:
#                     self.order = self.buy(size=self.size)
                    
    ## 日志记录
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print(f'{dt.isoformat()},{txt}')

    # 记录交易执行情况（可选，默认不输出结果）
    def notify_order(self, order):
        # 如果 order 为 submitted/accepted，返回空
        if order.status in [order.Submitted, order.Accepted]:
            return
        # 如果 order 为 buy/sell executed，报告价格结果
        if order.status in [order.Completed]: 
            if order.isbuy():
                self.log(f'买入：\n价格：%.2f,\
                交易金额：-%.2f,\
                手续费：%.2f' % (order.executed.price, order.executed.value, order.executed.comm))
                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:
                self.log(f'卖出:\n价格：%.2f,\
                交易金额：%.2f,\
                手续费：%.2f' % (order.executed.price, order.executed.price*self.size, order.executed.comm))
            self.bar_executed = len(self) 

        # 如果指令取消/交易失败, 报告结果
        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('交易失败')
        self.order = None

    # 记录交易收益情况（可省略，默认不输出结果）
    def notify_trade(self,trade):
        if not trade.isclosed:
            return
        self.log(f'策略收益\n毛收益 {trade.pnl:.2f}, 净收益 {trade.pnlcomm:.2f}')

    # 回测结束后输出结果（可省略，默认输出结果）
    def stop(self):
        self.log('期末总资金 %.2f' %
                 (self.broker.getvalue()), doprint=True)

### 运行回测任务

In [ ]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime
import os.path
import sys

import backtrader as bt

if __name__ == '__main__':
    # 创建 Cerebro 对象
    cerebro = bt.Cerebro()

    # 创建 Data Feed
    df_backtest.index = pd.to_datetime(df_backtest.index)
    start = df_backtest.index[0]
    end = df_backtest.index[-1]
    print(start, '-', end)
    data = bt.feeds.PandasData(dataname=df_backtest, fromdate=start, todate=end)
    
    # 将 Data Feed 添加至 Cerebro
    cerebro.adddata(data)

    # 添加策略 Cerebro
    cerebro.addstrategy(MyStrategy, 
                        model_path=model_path,
                        long_threshold=params["long_threshold"], 
                        short_threshold=params["short_threshold"], 
                        profit_target=params["profit_target"], 
                        stop_target=params["stop_target"],
                        repeat_count=params["repeat_count"], 
                        repeat_step=params["repeat_step"])
    
    # 设置初始资金
    cerebro.broker.setcash(100000.0)
    # 设置手续费为万二
    cerebro.broker.setcommission(commission=0.0002) 

    # 在开始时 print 初始账户价值
    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())

    # 运行回测流程
    cerebro.run()

    # 在结束时 print 最终账户价值
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

In [ ]:
import matplotlib
%matplotlib inline

# 画图并保存
fig = cerebro.plot(iplot=False)[0][0]
fig.set_size_inches(30, 18)
fig.savefig('{}/plot.png'.format(directory), dpi=100)